In [11]:
import nibabel as nib
import numpy as np
import os
import glob
import SimpleITK as sitk
from scipy.ndimage import zoom

In [21]:
def update_header(input_file, save_img=False, output_dir=None):
    file_name = os.path.basename(input_file)

    nib_file = nib.load(input_file)

    header = nib_file.header

    voxel_spacing = header.get_zooms()

    qoffset_x = header['qoffset_x']
    qoffset_y = header['qoffset_y']
    qoffset_z = header['qoffset_z']

    new_affine = np.array([
        [voxel_spacing[0], 0, 0, qoffset_x],
        [0, voxel_spacing[1], 0, qoffset_y],
        [0, 0, voxel_spacing[2], qoffset_z],
        [0, 0, 0, 1]
    ])

    new_header = header.copy()

    new_header['srow_x'] = new_affine[0, :]
    new_header['srow_y'] = new_affine[1, :]
    new_header['srow_z'] = new_affine[2, :]

    updated_image = nib.Nifti1Image(nib_file.get_fdata(), new_affine, header=new_header)

    if save_img:
        nib.save(updated_image, os.path.join(output_dir, 'header_updated'+ file_name))
    
    return updated_image, file_name

In [13]:
def down_sample(file_path, factor=5, save_img=False, output_dir=None):
    image = nib.load(file_path)
    file_name = os.path.basename(file_path)
    
    downsampled_image = image.slicer[::factor, :, :]
    
    if save_img:
        nib.save(downsampled_image, os.path.join(output_dir, 'downsampled_' + file_name))

    return downsampled_image
    

In [14]:
def resample_volume(volumne_path="", interpolator=sitk.sitkLinear, new_spacing=None, output_path=""):
    if new_spacing is None:
        new_spacing = [1, 1, 1]

    voulume = sitk.ReadImage(volumne_path, sitk.sitkFloat32)
    original_spacing = voulume.GetSpacing()
    original_size = voulume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)]
    resample_volume = sitk.Resample(voulume, new_size, sitk.Transform(), interpolator, voulume.GetOrigin(), new_spacing, voulume.GetDirection(), 0.0, voulume.GetPixelID())

    sitk.WriteImage(resample_volume, output_path)

In [23]:
def fix_dim(nifti_file, new_shape, save_img=False, output_dir=None):
    # Load NIfTI image
    img = nib.load(nifti_file)
    data = img.get_fdata()
    
    # Original shape and voxel dimensions
    original_shape = data.shape
    original_affine = img.affine
    
    # Calculate the rescaling factors for each dimension
    scaling_factors = [new_dim / old_dim for new_dim, old_dim in zip(new_shape, original_shape)]
    
    # Resample the image data to the new shape
    resampled_data = zoom(data, scaling_factors, order=1)  # 'order=1' for linear interpolation
    
    # Create new affine by scaling the voxel sizes
    new_affine = original_affine.copy()
    new_affine[:3, :3] *= np.array([original_shape[i] / new_shape[i] for i in range(3)])
    
    # Create new NIfTI image with resampled data and updated affine
    resampled_img = nib.Nifti1Image(resampled_data, new_affine)

    if save_img:
        nib.save(resampled_img, os.path.join(output_dir, 'dimfixed_' + os.path.basename(nifti_file)))
    
    return resampled_img

In [40]:
original_dataset = "DataSet"
header_updated = "Header-Updated"
resampled_highRes = "Resampled/High-Res"
resampled_lowRes = "Resampled/Low-Res"
downsampled = "Down-sampled"
dim_fixed = "Dim-Fixed"

In [25]:
original_data_paths = glob.glob(os.path.join(original_dataset, '*'))

In [26]:
sample_data_point = original_data_paths[0]

In [27]:
# Update header
updated_image, file_name = update_header(sample_data_point, save_img=True, output_dir=header_updated)

In [28]:
dim_fixed_image = fix_dim(sample_data_point, (150, 256, 256), save_img=True, output_dir=dim_fixed)

In [34]:
sample_data_point1 = original_data_paths[4]
dim_fixed_image1 = fix_dim(sample_data_point1, (150, 256, 256), save_img=True, output_dir=dim_fixed)

In [35]:
dim_fixed_data_paths = glob.glob(os.path.join(dim_fixed, '*'))
sample_data_point1 = dim_fixed_data_paths[4]
updated_image1, file_name1 = update_header(sample_data_point1, save_img=True, output_dir=header_updated)

In [38]:
header_updated_data_paths = glob.glob(os.path.join(header_updated, '*'))
sample_data_point1 = header_updated_data_paths[4]
print(os.path.basename(sample_data_point1))
# downsampled_image = down_sample(sample_data_point1, save_img=True, output_dir=downsampled)

header_updateddimfixed_CC0005_philips_15_62_M.nii.gz


In [41]:
downsampled_image = down_sample(sample_data_point1, save_img=True, output_dir=downsampled)

In [ ]:
print(downsampled_image.header)

In [ ]:
print(updated_image1.header)

In [44]:
# Resaple the High-Res image
resample_volume(sample_data_point1, output_path=os.path.join(resampled_highRes, os.path.basename(sample_data_point1)))

In [45]:
# Resaple the Low-Res image
downsampled_data_paths = glob.glob(os.path.join(downsampled, '*'))
sample_data_point_downsampled = downsampled_data_paths[0]
resample_volume(sample_data_point_downsampled, output_path=os.path.join(resampled_lowRes, os.path.basename(sample_data_point_downsampled)))

In [46]:
len(original_data_paths)

359